# Importing Libraries and Setup

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import google.generativeai
import gradio as gr
from IPython.display import display, update_display, Markdown
import google.generativeai

In [2]:
import subprocess
import sys
import io

In [37]:
import torch
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM, TextStreamer
from huggingface_hub import login , InferenceClient

# Setting up API's 

In [4]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
claude_api_key = os.getenv('ANTHROPIC_API_KEY')
gemini_api_key = os.getenv('GOOGLE_API_KEY')
hf_token = os.getenv('HF_API_KEY')

In [5]:
login(hf_token, add_to_git_credential = True)

In [6]:
openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()

## Prompts and Models Setup

In [7]:
claude_model = 'claude-sonnet-4-20250514'
gpt_model = 'gpt-5-2025-08-07'
gemini_model = 'gemini-2.5-pro'

In [8]:
system_message = """
You are an advanced code-generation AI specialized in algorithmic trading systems.

Your goal is to generate clean, correct, and production-ready Python code that can:
1. Fetch historical market data using **free, no-API-key data sources**:
   - Yahoo Finance via the 'yfinance' Python library
   - Stooq via direct CSV URLs
2. Parse OHLCV price data and prepare it for analysis.
3. Implement algorithmic trading strategies.
4. Simulate trades and track:
   - Buy price
   - Sell price
   - Quantity
   - Profit/Loss (PnL)
   - Running equity curve
5. Generate simple visualizations:
   - Price chart
   - Buy/Sell markers
   - Optional equity curve
6. Return a list of Trade objects where:
   - Trade(ticker, quantity) represents a BUY (> 0) or SELL (< 0).

---------------------------------------
TRADING FUNCTION REQUIREMENTS
---------------------------------------
You will be shown one example trading function.

You must generate **five additional trading functions**:
- trade2()
- trade3()
- trade4()
- trade5()
- trade6()

Each function must:
- Implement a **unique** trading strategy
  (examples: momentum, mean reversion, moving average cross, RSI-based, random, risk-managed)
- Use real price data (from yfinance or Stooq)
- Include clear comments explaining the strategy logic
- Return a list of Trade objects
- Contain ONLY Python code (no explanations)

---------------------------------------
DATA SOURCES ALLOWED
---------------------------------------
You must only use these free data sources:

1. **Yahoo Finance (recommended)**
   - Accessed via: import yfinance as yf
   - Does not require an API key
   - Provides historical OHLCV data
   - Suitable for backtesting and charting

2. **Stooq**
   - Direct CSV URL format:
     https://stooq.com/q/d/l/?s={TICKER}&i=d
   - No API key required
   - Good for daily OHLCV

No paid APIs, no API keys, no restricted services.

---------------------------------------
SIMULATION REQUIREMENTS
---------------------------------------
The generated Python code may need to simulate:
- Trade entries and exits
- PnL calculations
- Markers (buy/sell) for chart generation
- Optional equity curve

Keep simulation logic simple and clean.

---------------------------------------
OUTPUT RULES
---------------------------------------
- Your reply must contain ONLY Python code.
- No markdown, no explanation text, no commentary.
- Code must be executable as-is.
- Every function must return a list of Trade objects.

---------------------------------------
BEHAVIOR RULES
---------------------------------------
- Always generate safe, deterministic, bug-free code.
- Add comments inside the code explaining key logic.
- Avoid any external dependencies beyond:
   - yfinance
   - pandas
   - numpy
   - matplotlib
   - urllib / requests (optional for Stooq)
- Keep code readable and organized.
- Strategy variety is important—avoid repetition.

You are a senior trading-systems engineer. Produce code at that standard.
"""

In [9]:
def user_prompt_for(trade_function):
    """
    Constructs the user prompt that will be sent to the model.
    It appends the provided example function, followed by detailed instructions
    consistent with the system prompt requirements.
    """
    return f"""
Below is an example trading function that demonstrates the required format,
structure, API usage, and return type:

{trade_function}

--------------------------------------
TASK FOR THE MODEL
--------------------------------------
Using the example function as a reference, write **five additional trading functions**:
- trade2()
- trade3()
- trade4()
- trade5()
- trade6()

Each function must:

1. Use only **free, no-API-key data sources**:
   - Yahoo Finance via `yfinance`
   - Stooq via direct CSV URLs

2. Fetch and process real historical OHLCV data for the tickers provided.

3. Implement **unique and distinct trading strategies**, such as:
   - Momentum / trend-following
   - Mean reversion
   - Moving average crossovers
   - RSI or indicator-based strategies
   - Volatility or risk-managed strategies
   - Randomized or Monte Carlo–style approaches

4. Include **clear comments inside the code** explaining:
   - The strategy logic  
   - Buy/sell conditions  
   - Any indicator calculations  
   - How decisions are made  

5. Return a list of `Trade` objects, where:
   - Trade(ticker, quantity) indicates BUY (> 0) or SELL (< 0).

6. Produce **ONLY Python code** in the output.
   - No markdown
   - No explanations
   - No additional text

Follow the structure of the example function and maintain clean, readable code.
"""

In [10]:
trading_function_example =  """
# tickers is a list of stock tickers (strings)
import tickers

# prices is a dict; the key is a ticker and the value is a list of historic prices, today first
import prices

# Trade represents a decision to buy or sell a quantity of a ticker
# Trade("IBM", 100) for a trade object representing purchasing 100 shares of IBM stock
# Trade("IBM", -50) for a trade object representing selling or shorting 50 shares of IBM stock
import Trade

import random
import numpy as np

def trade1():
    # Buy top performing stock in the last 5 days
    avg_prices = {ticker: np.mean(prices[ticker][:5]) for ticker in tickers}
    best_ticker = max(avg_prices, key=avg_prices.get)
    trade = Trade(best_ticker, 100)
    return [trade]
"""

In [11]:
def messages_for(trade_function):
    messages = [
        {'role':'system', 'content':system_message},
        {'role':'user','content':user_prompt_for(trade_function)}
    ]
    return messages

# Frontier Models Streaming:

In [12]:
def stream_gpt(trade_function):
    response = openai.chat.completions.create(
        model = gpt_model,
        messages = messages_for(trade_function),
        stream = True
    )
    result = ''
    for chunk in response:
        result += chunk.choices[0].delta.content or ''
        result = result.replace('```','').replace('markdown','').replace("```python\n", "")
        yield result

In [13]:
def stream_claude(trade_function):
    response = claude.messages.stream(
        model = claude_model,
        messages = [{'role':'user','content':user_prompt_for(trade_function)}],
        system = system_message,
        max_tokens = 5000
    )
    result = ''
    with response as stream:
        for text in stream.text_stream:
            result += text
            result = result.replace('```','').replace('markdown','').replace("```python\n", "")
            yield result

In [14]:
def stream_gemini(trade_function):
    gemini = google.generativeai.GenerativeModel(
        model_name = gemini_model, 
        system_instruction = system_message
    )
    response = gemini.generate_content(user_prompt_for(trade_function),stream = True)
    result = ''

    for chunk in response:
        if chunk.text:
            result += chunk.text or ''
            result = result.replace('```','').replace('markdown','').replace("```python\n", "")
            yield result

# Open Sourced Model Streaming:

In [51]:
code_qwen = 'Qwen/CodeQwen1.5-7B-Chat'

code_gemma = 'google/codegemma-7b-it'
code_gemma_url = 'https://l77mnbo5de1yfudo.us-east-1.aws.endpoints.huggingface.cloud'

code_llama = 'codellama/CodeLlama-70b-Instruct-hf'
code_llama_url = 'https://qzziiwlqibu37ekc.us-east-2.aws.endpoints.huggingface.cloud'

In [52]:
quant_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_quant_type = 'nf4'
)

In [54]:
def stream_qwen(trade_function):
    messages = messages_for(trade_function)
    tokenizer = AutoTokenizer.from_pretrained(code_qwen, trust_remote_code = True)
    tokenizer.pad_token = tokenizer.eos_token

    inputs = tokenizer.apply_chat_template(messages, return_tensors = 'pt', add_generation_prompt = True).to('mps')
    model = AutoModelForCausalLM.from_pretrained(
        code_qwen,
        quantization_config = quant_config,
        device_map="mps",        
        torch_dtype=torch.float16,
        use_cache=True
    )

    streamer = TextStreamer(tokenizer)
    output = model.generate(inputs, streamer = streamer, max_new_tokens = 5000)
    result = tokenizer.decode(output[0], skip_special_tokens = True)
    yield result

In [55]:
def stream_gemma(trade_function):
    messages = [{'role':'user','content':user_prompt_for(trade_function)}]
    tokenizer = AutoTokenizer.from_pretrained(code_gemma, trust_remote_code= True)
    tokenizer.pad_token = tokenizer.eos_token

    inputs = tokenizer.apply_chat_template(messages , tokenize = False, add_generation_prompt = True)

    client = InferenceClient(code_gemma_url, token = hf_token)
    stream = client.text_generation(inputs, stream = True, details = True, max_new_tokens = 3000)

    result = ''
    for chunk in stream:
        result += chunk.token.text or ''
        yield result

In [56]:
def optimize(model, trade_function):
    if model == 'GPT':
        result = stream_gpt(trade_function)
    elif model == 'Claude':
        result = stream_claude(trade_function)
    elif model == 'Gemini':
        result = stream_gemini(trade_function)
    elif model == 'Qwen':
        result = stream_qwen(trade_function)
    elif model == 'Gemma':
        result = stream_gemma(trade_function)
    else:
        raise ValueError('Unknown Model')

    for chunk in result:
        yield chunk

In [57]:
with gr.Blocks() as ui:
    gr.Markdown('## Trading Advice Simulator')
    with gr.Row():
        trade1_input = gr.Textbox(label ='Trade 1', lines = 20, value =trading_function_example)
        ouput_trade = gr.Textbox(label = 'Generated Trading Decisions', lines = 20)
    with gr.Row():
        model = gr.Dropdown(['GPT','Claude','Gemini','Qwen','Gemma'],label = 'Select Model')
    with gr.Row():
        generate_btn = gr.Button('Simulate Trading')

    generate_btn.click(fn=optimize,inputs=[model, trade1_input], outputs=[ouput_trade])
ui.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
